In [ ]:

# Gurobi is a powerful optimization solver that excels at handling linear programming, mixed-integer programming, and 
# quadratic programming problems. 

# Imagine a manufacturing company with multiple products, production lines, and resource constraints. The goal is to 
# maximize profit while considering constraints on production capacities, raw material availability, and labor hours.


In [40]:

from gurobipy import GRB, Model
import pandas as pd

# Sample DataFrame
data = {
    'Product': ['A', 'B', 'C'],
    'Profit': [10, 15, 20],
    'Capacity': [100, 150, 200],
    'Labor_Hours': [2, 3, 4]
}

df = pd.DataFrame(data)
df


,Product,Profit,Capacity,Labor_Hours
0,A,10,100,2
1,B,15,150,3
2,C,20,200,4


In [41]:

products = df['Product'].tolist()

def calculate_total_profit(production_quantities, df):
    return sum(production_quantities[product] * df.loc[df['Product'] == product, 'Profit'].values[0] for product in products)


def print_scenario_results(header, production_quantities, df):
    print(f"\n{header}:")
    total_profit = calculate_total_profit(production_quantities, df)
    for product in products:
        print(f"{product}: {production_quantities[product]}")
    print(f"Total Profit: {total_profit}")
    return total_profit



# Set arbitrary production scenario
arbitrary_production = {'A': 30, 'B': 20, 'C': 10}  

# Calculate arbitrary profit
arbitrary_profit = sum(arbitrary_production[product] * df.loc[df['Product'] == product, 'Profit'].values[0] for product in products)
arbitrary_profit


800

In [42]:

# Create Gurobi model
model = Model("production_model")
production = model.addVars(products, name="production", vtype=GRB.INTEGER)

# Set objective function
model.setObjective(production.prod(df.set_index('Product')['Profit'].to_dict()), sense=GRB.MAXIMIZE)

# Add constraints
model.addConstr(production.prod(df.set_index('Product')['Capacity'].to_dict()) <= 10000, "capacity_constraint")
model.addConstr(production.prod(df.set_index('Product')['Labor_Hours'].to_dict()) <= 250, "labor_hours_constraint")

# Optimize the model
model.optimize()


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 13th Gen Intel(R) Core(TM) i5-1335U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 2 rows, 3 columns and 6 nonzeros
Model fingerprint: 0xb2526335
Variable types: 0 continuous, 3 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e+00, 2e+02]
  Objective range  [1e+01, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 1e+04]
Found heuristic solution: objective 1000.0000000
Presolve removed 1 rows and 1 columns
Presolve time: 0.00s
Presolved: 1 rows, 2 columns, 2 nonzeros
Variable types: 0 continuous, 2 integer (0 binary)

Root relaxation: cutoff, 0 iterations, 0.00 seconds (0.00 work units)

Explored 1 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 12 (of 12 available processors)

Solution count 1: 1000 

Optimal solution found (tolerance 1.00e-

In [43]:

# Print the results for both scenarios and calculate overall savings
arbitrary_total_profit = print_scenario_results("Arbitrary Scenario", arbitrary_production, df)

if model.status == GRB.OPTIMAL:
    optimized_production = {product: production[product].x for product in products}
    optimized_total_profit = print_scenario_results("Optimized Solution", optimized_production, df)

    savings_percentage = (((optimized_total_profit / arbitrary_total_profit)) * 100) if arbitrary_total_profit > 0 else 0
    print(f"\nOverall Savings: {savings_percentage:.2f}%")
else:
    print("No optimal solution found.")
    


Arbitrary Scenario:
A: 30
B: 20
C: 10
Total Profit: 800

Optimized Solution:
A: 100.0
B: -0.0
C: -0.0
Total Profit: 1000.0

Overall Savings: 125.00%


In [3]:

# This script creates an optimization model, defines decision variables, sets the objective function, adds constraints, and 
# then solves the model using Gurobi. The results are then displayed, including the optimal quantities and the maximum 
# total profit.

# The objective function in an optimization problem defines the quantity that the optimization process seeks to either 
# maximize or minimize. In the context of the production planning problem using Gurobi, the objective function is 
# formulated to maximize the total profit generated by producing different products on various production lines.

# In optimization modeling, you typically set constraints based on the maximum capacity or limitations of your resources, 
# not by summing up the individual values. The individual values represent the capacity or resource usage for each unit 
# of the respective product. In a real-world scenario, constraints should be based on the actual limitations and capacities 
# of the resources involved, such as the available production capacity and labor hours.
